In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import random
from pymongo import MongoClient
import copy

from state import getStaticInfo, addIndexInfo, convertToStateVector, getQueryMetrics, saveAsJSON
from index import create_single_field_index, delete_single_field_index, reset_index_config

In [2]:
class Actor(nn.Module):
    def __init__(self, state_size, action_size):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(state_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, action_size)
        self.init_weights()

    def init_weights(self):
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.xavier_uniform_(self.fc3.weight)
        nn.init.zeros_(self.fc1.bias)
        nn.init.zeros_(self.fc2.bias)
        nn.init.zeros_(self.fc3.bias)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))

        # action_probs = torch.sigmoid(self.fc3(x))
        
        output = self.fc3(x)
        field = torch.sigmoid(output[:-1])
        index_state = torch.tanh(output[-1])
        index_state = index_state.unsqueeze(0)
        action_probs = torch.cat((field, index_state))

        return action_probs

In [3]:
class Critic(nn.Module):
    def __init__(self, state_size):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(state_size, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, 1)
        self.init_weights()

    def init_weights(self):
        nn.init.xavier_uniform_(self.fc1.weight)
        nn.init.xavier_uniform_(self.fc2.weight)
        nn.init.xavier_uniform_(self.fc3.weight)
        nn.init.zeros_(self.fc1.bias)
        nn.init.zeros_(self.fc2.bias)
        nn.init.zeros_(self.fc3.bias)

    def forward(self, state):
        x = torch.relu(self.fc1(state))
        x = torch.relu(self.fc2(x))
        value = self.fc3(x)
        return value

In [12]:
state_size = 192
action_size = 12
learning_rate = 0.0001
actor = Actor(state_size, action_size).cuda()
critic = Critic(state_size).cuda()

In [13]:
actor_optimizer = optim.Adam(actor.parameters(), lr=learning_rate)
critic_optimizer = optim.Adam(critic.parameters(), lr=learning_rate)

In [ ]:
#t ocontinue training using an existing model
actor.load_state_dict(torch.load('actor_model.pth'))
critic.load_state_dict(torch.load('critic_model.pth'))

In [14]:
client = MongoClient('mongodb://localhost:27017/')
db_conn = client['benchmark_db1']

In [ ]:
print("collecting static info...")
partial_state = copy.deepcopy(getStaticInfo(db_conn))
og_state_dict = copy.deepcopy(addIndexInfo(db_conn, partial_state))
saveAsJSON(og_state_dict)
og_state_vector, collection_list, field_list = convertToStateVector(og_state_dict) # returns a 1X192 list

In [15]:
import json
with open('state.json', 'r') as file:
    og_state_dict = json.load(file)
og_state_vector, collection_list, field_list = convertToStateVector(og_state_dict)

In [101]:
def env_step_og(action, state, state_dict):
    collectionIdx, fieldIdx, indexed = action[0], action[1], action[2]
    collection_name = collection_list[collectionIdx]
    field_name = field_list[collectionIdx][fieldIdx]
    if fieldIdx < len(field_list[collectionIdx]):
        print("Action : ", collection_list[collectionIdx], field_list[collectionIdx][fieldIdx], indexed)
        if state_dict[collection_name][field_name]['cardinality'] > 0.5:
            if indexed > 0:
                res = create_single_field_index(db_conn, collection_list[collectionIdx], field_list[collectionIdx][fieldIdx])
                if res == 0: # trying to index and already indexed field
                    reward = -99999999
                else :
                    reward = getQueryMetrics(db_conn)['executionTimeMillis']
                    state_dict = addIndexInfo(db_conn, state_dict)
                    state, _, _ = convertToStateVector(state_dict)
            elif indexed < 0 :
                res = delete_single_field_index(db_conn, collection_list[collectionIdx], field_list[collectionIdx][fieldIdx])
                if res == 0: # trying to delete index from an field that is not indexed
                    reward = -99999999
                else :
                    reward = getQueryMetrics(db_conn)['executionTimeMillis']
                    state_dict = addIndexInfo(db_conn, state_dict)
                    state, _, _ = convertToStateVector(state_dict)
            else:
                reward = getQueryMetrics(db_conn)['executionTimeMillis']
        else :
            reward = -99999999
            state_dict[collection_name][field_name][indexed] = 1
            state_dict = addIndexInfo(db_conn, state_dict)
            state, _, _ = convertToStateVector(state_dict)
    else :
        reward = -99999999
        state[collectionIdx*6+fieldIDx] = 1
    print(reward)
    return reward, state, state_dict

In [16]:
def env_step(action, state, state_dict):
    collectionIdx, fieldIdx, indexed = action[0], action[1], action[2]
    
    # Validate indices
    if collectionIdx >= len(collection_list) or fieldIdx >= len(field_list[collectionIdx]):
        reward = -99999999
        state[collectionIdx * 6 + fieldIdx] = 1  # Mark invalid action in the state
        print(reward)
        return reward, state, state_dict

    collection_name = collection_list[collectionIdx]
    field_name = field_list[collectionIdx][fieldIdx]

    print("Action : ", collection_list[collectionIdx], field_list[collectionIdx][fieldIdx], indexed)
    
    # Check if the field's cardinality is greater than 0.5
    if state_dict[collection_name][field_name]['cardinality'] > 0.5:
        if indexed > 0:
            # Try to create an index
            res = create_single_field_index(db_conn, collection_list[collectionIdx], field_list[collectionIdx][fieldIdx])
            if res == 0:  # Index already exists
                reward = -99999999
            else:
                # Successful indexing, calculate reward
                reward = -getQueryMetrics(db_conn)['executionTimeMillis']
                state_dict = addIndexInfo(db_conn, state_dict)
                state, _, _ = convertToStateVector(state_dict)
        elif indexed < 0:
            # Try to delete an index
            res = delete_single_field_index(db_conn, collection_list[collectionIdx], field_list[collectionIdx][fieldIdx])
            if res == 0:  # Index doesn't exist
                reward = -99999999
            else:
                # Successful deletion, calculate reward
                reward = -getQueryMetrics(db_conn)['executionTimeMillis']
                state_dict = addIndexInfo(db_conn, state_dict)
                state, _, _ = convertToStateVector(state_dict)
        else:
            # No change, calculate reward based on current state
            reward = getQueryMetrics(db_conn)['executionTimeMillis']
    else:
        # Punish actions on fields with cardinality <= 0.5
        reward = -99999999
        state_dict = addIndexInfo(db_conn, state_dict)
        state, _, _ = convertToStateVector(state_dict)

    print(reward)
    return reward, state, state_dict

In [17]:
def train_step(state, reward, next_state, done):
    state = torch.tensor(state, dtype=torch.float32).cuda()
    next_state = torch.tensor(next_state, dtype=torch.float32).cuda()
    reward = torch.tensor(reward, dtype=torch.float32).cuda()
    done = torch.tensor(done, dtype=torch.float32).cuda()

    # Critic update
    value = critic(state)
    next_value = critic(next_state)
    target = reward + (1 - done) * 0.99 * next_value
    critic_loss = nn.functional.mse_loss(value, target.detach())
    critic_optimizer.zero_grad()
    critic_loss.backward()
    critic_optimizer.step()

    # Actor update
    action_probs = actor(state)
    action_log_probs = torch.log(action_probs)
    advantage = (target - value).detach()
    actor_loss = -action_log_probs * advantage
    actor_optimizer.zero_grad()
    actor_loss.mean().backward()
    actor_optimizer.step()

In [ ]:
num_episodes = 1000
max_steps_per_episode = 100
for episode in range(num_episodes):
    state_dict = copy.deepcopy(og_state_dict)
    state = og_state_vector[:]

    # print(f"Episode {episode}: Initial state size: {len(state)}")
    done = False
    step_count = 0

    while not done:
        action_probs = actor(torch.tensor(state, dtype=torch.float32).cuda())
        try:
            collection = torch.argmax(action_probs[:5]).item()
            field = torch.argmax(action_probs[5:11]).item()
            index_status = round(action_probs[11].item()) #action_probs[11].item()
            print(collection+1, field+1, action_probs[11].item())
        except Exception as e:
            print(f"Error at Episode {episode}, Step {step_count}: {e}")
            print(f"action_probs: {action_probs}")
            break

        action = (collection, field, index_status)
        reward, next_state, next_state_dict = env_step(action, state, state_dict)
        # next_state = og_state_vector
 
        train_step(state, reward, next_state, done)
        state = next_state

        step_count += 1
        if step_count >= max_steps_per_episode:
            done = True

        if episode%1 == 0 and step_count == max_steps_per_episode:
            print(f"Episode {episode} action_probs = {action_probs}")

    reset_index_config(db_conn, og_state_dict)

In [ ]:
torch.save(actor.state_dict(), 'actor_model.pth')
torch.save(critic.state_dict(), 'critic_model.pth')

In [13]:
def predict_action(state):
    actor.load_state_dict(torch.load('actor_model.pth'))
    state = torch.tensor(state, dtype=torch.float32).cuda()
    action_probs = actor(state)
    collection = torch.argmax(action_probs[:5]).item()
    field = torch.argmax(action_probs[5:11]).item()
    index_status = action_probs[11].item()
    return collection, field, index_status

In [ ]:
print(og_state_dict['collection_3']['currency'])
reset_index_config(db_conn, dict(og_state_dict))